# Refazendo imports e manipulação dos dados

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


dataset = pd.read_csv('db/movie_dataset.csv', sep=',',  index_col=0, na_values='?')

dataset = dataset.loc[:, ~dataset.columns.isin(['original_title', 'overview', 'tagline', 'title', 'homepage', 'crew', 'keywords', 'original_language', 'id'])]
dataset = dataset.loc[dataset['status'] == 'Released']

dataset_base = dataset.loc[:, dataset.columns.isin(['budget', 'revenue', 'popularity', 'runtime', 'vote_average', 'vote_count'])]
noise = np.random.randint(0, 101, size=dataset_base['revenue'].shape[0])
dataset_base['revenue'] = dataset_base['revenue'] + noise

dataset_base_rev_class = pd.qcut(dataset_base['revenue'], q=4, labels=['very low', 'low', 'medium', 'high'])
dataset_base['revenue'] = dataset_base_rev_class

dataset_dummies = dataset
dataset_dummies = pd.get_dummies(dataset_dummies)
dataset_dummies['revenue'] = dataset_dummies['revenue'] +noise
dataset_dummies_rev = pd.qcut(dataset_dummies['revenue'], q=4, labels=['very low', 'low', 'medium', 'high'])
dataset_dummies['revenue'] = dataset_dummies_rev

/tmp/ipykernel_35560/3284943925.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_base['revenue'] = dataset_base['revenue'] + noise
/tmp/ipykernel_35560/3284943925.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_base['revenue'] = dataset_base_rev_class


# Grid Search K-Neighbours

Primeiro fazemos a execução com o dataset utilizado de base

### Dataset base

In [3]:
dataset_base = dataset_base.loc[:, dataset_base.columns != 'vote_average']
dataset_base = dataset_base.loc[:, dataset_base.columns != 'runtime']

rev_pred_paramX = dataset_base.loc[:, dataset_base.columns != 'revenue'] 
rev_pred_paramY = np.array(dataset_base.loc[:, dataset_base.columns == 'revenue']).ravel()

X_train, X_test, y_train, y_test = train_test_split(rev_pred_paramX, rev_pred_paramY, test_size=0.10, stratify=rev_pred_paramY, random_state=42)

rev_pred_paramX.fillna(1, inplace=True)
rev_pred_paramY = np.nan_to_num(rev_pred_paramY, nan=1)

/tmp/ipykernel_35560/124007373.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_pred_paramX.fillna(1, inplace=True)


In [4]:
parameters = {'n_neighbors' : [1, 3, 5, 7, 9, 11, 13], 'weights' : ['uniform', 'distance']}

knn = KNeighborsClassifier()

gs = GridSearchCV(knn, parameters, cv=10, scoring='accuracy')
gs.fit(rev_pred_paramX, rev_pred_paramY)

results = pd.DataFrame(gs.cv_results_)
view = ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
results[view].sort_values(by='rank_test_score')

,params,mean_test_score,std_test_score,rank_test_score
4,"{'n_neighbors': 5, 'weights': 'uniform'}",0.361574,0.056746,1
2,"{'n_neighbors': 3, 'weights': 'uniform'}",0.354291,0.044727,2
6,"{'n_neighbors': 7, 'weights': 'uniform'}",0.352818,0.050253,3
0,"{'n_neighbors': 1, 'weights': 'uniform'}",0.351995,0.044801,4
1,"{'n_neighbors': 1, 'weights': 'distance'}",0.351995,0.044801,4
5,"{'n_neighbors': 5, 'weights': 'distance'}",0.351780,0.049601,6
8,"{'n_neighbors': 9, 'weights': 'uniform'}",0.350926,0.071579,7
10,"{'n_neighbors': 11, 'weights': 'uniform'}",0.349878,0.076113,8
3,"{'n_neighbors': 3, 'weights': 'distance'}",0.349279,0.051632,9
7,"{'n_neighbors': 7, 'weights': 'distance'}",0.347607,0.049710,10


Esse algoritmo apresenta cerca de 36% de acurácia e leva pouco mais de 1.3s para ser treinado

### Dataset com dummies

In [5]:
dataset_dummies = dataset_dummies.loc[:, dataset_dummies.columns != 'vote_average']
dataset_dummies = dataset_dummies.loc[:, dataset_dummies.columns != 'runtime']

rev_pred_paramX = dataset_dummies.loc[:, dataset_dummies.columns != 'revenue'] 
rev_pred_paramY = np.array(dataset_dummies.loc[:, dataset_dummies.columns == 'revenue']).ravel()

X_train, X_test, y_train, y_test = train_test_split(rev_pred_paramX, rev_pred_paramY, test_size=0.10, stratify=rev_pred_paramY, random_state=42)

rev_pred_paramX.fillna(1, inplace=True)
rev_pred_paramY = np.nan_to_num(rev_pred_paramY, nan=1)

/tmp/ipykernel_35560/727090146.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_pred_paramX.fillna(1, inplace=True)


In [6]:
parameters = {'n_neighbors' : [1, 3, 5, 7, 9, 11, 13], 'weights' : ['uniform', 'distance']}

knn = KNeighborsClassifier()

gs = GridSearchCV(knn, parameters, cv=10, scoring='accuracy')
gs.fit(rev_pred_paramX, rev_pred_paramY)

results = pd.DataFrame(gs.cv_results_)
view = ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
results[view].sort_values(by='rank_test_score')

,params,mean_test_score,std_test_score,rank_test_score
4,"{'n_neighbors': 5, 'weights': 'uniform'}",0.360527,0.060720,1
5,"{'n_neighbors': 5, 'weights': 'distance'}",0.358245,0.048136,2
7,"{'n_neighbors': 7, 'weights': 'distance'}",0.355741,0.049435,3
6,"{'n_neighbors': 7, 'weights': 'uniform'}",0.351775,0.051252,4
0,"{'n_neighbors': 1, 'weights': 'uniform'}",0.351576,0.044104,5
1,"{'n_neighbors': 1, 'weights': 'distance'}",0.351576,0.044104,5
9,"{'n_neighbors': 9, 'weights': 'distance'}",0.351569,0.050619,7
11,"{'n_neighbors': 11, 'weights': 'distance'}",0.350942,0.051774,8
3,"{'n_neighbors': 3, 'weights': 'distance'}",0.350532,0.048675,9
8,"{'n_neighbors': 9, 'weights': 'uniform'}",0.350300,0.072058,10


Por outro lado, o dataset com dummies tem uma acurácia de 36% e leva em média 3 minuto e 10 segundos para o treinamento e execução